In [21]:
import pandas as pd
import logging 
import numpy as np
import re 
from IPython.display import display
from sqlalchemy import MetaData, Table
from database_utils import DatabaseConnector
from data_extraction import DataExtractor

In [13]:
instance = DataExtractor()

        # Read data from the 'legacy_users' table
df = instance.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

        

retrieve_pdf_data is working


In [16]:
df.info() 

display(len((df['card_number'].unique()))) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB


15299

In [20]:

# Ensure all card numbers are strings
df['card_number'] = df['card_number'].astype(str)

# Calculate the lengths of the card numbers
df['length'] = df['card_number'].apply(len)

# Group by length and show 10 examples of each group
length_groups = df.groupby('length').apply(lambda x: x.head(10)).reset_index(drop=True)

#import ace_tools as tools; tools.display_dataframe_to_user(name="Length Groups of Card Numbers", dataframe=length_groups)

length_groups

/var/folders/ps/4kr9fxh547bg4xm51tqj617h0000gn/T/ipykernel_5222/3510606622.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  length_groups = df.groupby('length').apply(lambda x: x.head(10)).reset_index(drop=True)


,card_number,expiry_date,card_provider,date_payment_confirmed,length
0,NULL,NULL,NULL,NULL,4
1,NULL,NULL,NULL,NULL,4
2,NULL,NULL,NULL,NULL,4
3,NULL,NULL,NULL,NULL,4
4,NULL,NULL,NULL,NULL,4
...,...,...,...,...,...
99,????3556268655280464,11/29,JCB 16 digit,1997-01-03,20
100,?4222069242355461965,12/23,VISA 19 digit,1999-01-25,20
101,????3512756643215215,09/30,JCB 16 digit,2008-07-26,20
102,????3543745641013832,01/24,JCB 16 digit,2004-06-21,20
